Instantiate an Athlete object with user data

In [1]:
import os
os.chdir('/Users/ericchuu/fit-af/src')
from dotenv import load_dotenv
from athlete.Athlete import Athlete
import json
import logging
logging.basicConfig(level=logging.INFO)
load_dotenv() # Load credentials from .env file

email = os.getenv('GARMIN_EMAIL')
password = os.getenv('GARMIN_PASSWORD')
intervals_icu_id = os.getenv('INTERVALS_ICU_ID')

user_prompt = '''
{
  "sex": "male",
  "age": 28,
  "goal_event": {
    "event": 5000,
    "goal_time": {
      "hours": 0,
      "minutes": 18,
      "seconds": 0
    }
  },
  "timeline_weeks": 2,
  "recent_time_trial": {
    "event": 5000,
    "hours": 0,
    "minutes": 20,
    "seconds": 0
  },
  "training_days_per_week": 4,
  "start_date": "2025-02-10"
}
'''
user_data = json.loads(user_prompt)

athlete = Athlete(
    user_id=1,
    email=email,
    password=password,
    intervals_icu_id=intervals_icu_id,
    training_plan_input=user_data,
    training_plan_dir="/Users/ericchuu/training_plan"
  )

/Users/ericchuu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from utils.pace_zones import calculate_zones_from_json
print(json.dumps(calculate_zones_from_json(user_data), indent=2))

{
  "vdot": 49.806233428066335,
  "zones": {
    "Easy": [
      3.39614006612044,
      2.83304760488861
    ],
    "Marathon": [
      3.761946332077611,
      3.433054543992328
    ],
    "Threshold": [
      3.9062447743897972,
      3.7256942266166315
    ],
    "Interval": [
      4.332543837184274,
      4.156096515701472
    ],
    "Repetition": [
      5.0224764432528985,
      4.507362163321937
    ]
  },
  "time_trial_speed": 4.166666666666667,
  "goal_event_speed": 4.62962962962963
}


Generate a training plan

In [3]:
athlete.generate_training_plan()

In [4]:
print(json.dumps(athlete.current_plan.model_dump(), indent=2))

{
  "plan_duration": {
    "value": 2.0,
    "unit": "weeks"
  },
  "athlete_level": "intermediate",
  "primary_goal": "Achieve a 5000m goal time of 18:00",
  "weeks": [
    {
      "week_number": 1,
      "start_date": "2025-02-10",
      "end_date": "2025-02-16",
      "area_of_focus": "vo2_max_development",
      "total_distance": {
        "value": 32000.0,
        "unit": "meters"
      },
      "total_time": {
        "value": 9000.0,
        "unit": "seconds"
      },
      "workouts": [
        {
          "workout_type": "Run",
          "workout_subtype": [
            "vo2max_intervals",
            "speed_intervals"
          ],
          "scheduled_date": "2025-02-10",
          "total_distance": {
            "value": 6000.0,
            "unit": "meters"
          },
          "estimated_duration": {
            "value": 1800.0,
            "unit": "seconds"
          },
          "terrain": "track",
          "phases": [
            {
              "type": "warmup",
    

In [ ]:
with open("/Users/ericchuu/training_plan/current_plan_1.json", "r") as f:
    training_plan = f.read()
print(training_plan)


In [6]:
from schemas.training_plan import TrainingPlan
workouts = athlete.generate_workout_files(TrainingPlan.model_validate_json(training_plan))

In [7]:
uploaded_workout_ids = athlete.upload_workout_files()